### Shared Word

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
lang_folder = "Turkish"  # Primary language
lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [4]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [5]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Arabic (ara)_Etymologeek_Result_All.xlsx']

In [6]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [7]:
word_list = df_pair["dict_entry_main"].values.tolist()

In [40]:
# Select N Gram Type
sent_ngram_file = "Sentence" # Two, Three, Four, Five, Sentence
 
if sent_ngram_file.lower() != "sentence":
    data_kind = f"{sent_ngram_file}gram"  # Twogram, Threegram, Fourgram, Fivegram
    folder_path = "N Gram"
else:
    data_kind = "sentence"
    folder_path = "Sentence"

In [41]:
df = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/{folder_path.capitalize()}/Merge/{data_kind.capitalize()}_Merge.csv")
#df = df.head(100000)
df

,sentence,frequency,code
0,evet,1948596,G1-L4-F1948596-0
1,fikret,1533918,G1-L6-F1533918-1
2,hayır,1250401,G1-L5-F1250401-2
3,tamam,882921,G1-L5-F882921-3
4,ne,753105,G1-L2-F753105-4
...,...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6,G6-L35-F6-2913965
2913966,eminim uydurmuştur,6,G2-L18-F6-2913966
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6,G9-L49-F6-2913967
2913968,tabii gerçek bir profesyonel o,6,G5-L30-F6-2913968


In [42]:
d_list  = df.iloc[:,0].values.tolist()

In [43]:
resultlist = []

In [44]:
#import os
#from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
resultlist = manager.list()

def word_in_wordgroup(d_list):
    mergelist = []
    try:
        word = d_list.split()
    except:
        pass
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist.append(word[j])
            if len(mergelist) == len(word):
                    resultlist.append(d_list)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list) # string_word liste

In [45]:
result_list = list(resultlist)
df_result = pd.DataFrame(result_list)
df_result = df_result.rename(columns = {0: f"{data_kind.lower()}"})
df_merge = pd.merge(df_result, df, how="left", on=f"{data_kind.lower()}")
df_merge_result = df_merge.sort_values(by="frequency", ascending=False)
df_merge_result.drop_duplicates(inplace=True)
df_merge_result[f"{data_kind.lower()}"] = df_merge_result[f"{data_kind.lower()}"].apply(lambda x: remove_repetition(x))  # added remove_repetition 
df_merge_result = df_merge_result[df_merge_result[f"{data_kind.lower()}"] != "repetitive_word_group"]  # remove_repetition part
df_merge_result.reset_index(drop=True, inplace=True)
df_merge_result

,sentence,frequency,code
0,hayır lütfen,7187,G2-L12-F7187-697
1,selam millet,5303,G2-L12-F5303-993
2,elbette hayır,4595,G2-L13-F4595-1167
3,aynı şey,2253,G2-L8-F2253-2436
4,lütfen hayır,2179,G2-L12-F2179-2508
...,...,...,...
712,tamam kayıt,6,G2-L11-F6-2851128
713,mesela aile,6,G2-L11-F6-2853842
714,tamam tamam elbette,6,G3-L19-F6-2855577
715,mavi kırmızı,6,G2-L12-F6-2899421


In [46]:
df_merge_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_{data_kind.capitalize()}.xlsx", index=False)

#### Concat Result

In [55]:
df_word = df_pair.loc[:,["dict_entry_main"]]
df_word.rename(columns={"dict_entry_main":"word"}, inplace=True)
df_word

,word
0,mıknatıs
1,mısdak
2,şey
3,şeytan
4,şeytanî
...,...
362,avrat
363,molla
364,rubai
365,كتاب


In [56]:
df_shared_twogram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Twogram.xlsx")
df_shared_twogram = df_shared_twogram.loc[:,["twogram","frequency"]]
df_shared_twogram.rename(columns={"frequency":"freq_twogram"}, inplace=True)
df_shared_twogram

,twogram,freq_twogram
0,hayır lütfen,11636
1,aynı şey,11019
2,kadar basit,10150
3,kadar emin,5919
4,selam millet,5475
...,...,...
3411,yani nefes,6
3412,kadar ifade,6
3413,yani maalesef,6
3414,rahat şarkı,6


In [57]:
df_shared_threegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Threegram.xlsx")
df_shared_threegram = df_shared_threegram.loc[:,["threegram","frequency"]]
df_shared_threegram.rename(columns={"frequency":"freq_threegram"}, inplace=True)
df_shared_threegram

,threegram,freq_threegram
0,hayır hayır lütfen,1498
1,hayır lütfen hayır,392
2,hayır lütfen lütfen,356
3,hayır elbette hayır,345
4,tekrar tekrar aynı,195
...,...,...
148,hayır hayır kabul,10
149,hayır hayır mühim,10
150,evlat aynı şey,10
151,elbette asla emin,10


In [58]:
df_shared_fourgram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fourgram.xlsx")
df_shared_fourgram = df_shared_fourgram.loc[:,["fourgram","frequency"]]
df_shared_fourgram.rename(columns={"frequency":"freq_fourgram"}, inplace=True)
df_shared_fourgram

,fourgram,freq_fourgram
0,hayır hayır hayır lütfen,165
1,hayır hayır lütfen lütfen,43
2,hayır lütfen lütfen lütfen,28
3,hayır mesela yani tamam,21
4,lütfen hayır hayır hayır,18
5,tamam tamam tamam tekrar,12
6,şeker adam şeker adam,12
7,saat saat dakika dakika,12
8,aynı şey tekrar tekrar,11
9,yani tekrar tekrar aynı,11


In [59]:
df_shared_fivegram = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Fivegram.xlsx")
df_shared_fivegram = df_shared_fivegram.loc[:,["fivegram","frequency"]]
df_shared_fivegram.rename(columns={"frequency":"freq_fivegram"}, inplace=True)
df_shared_fivegram

,fivegram,freq_fivegram
0,hayır hayır hayır lütfen lütfen,20
1,hayır lütfen lütfen lütfen lütfen,10


In [60]:
df_shared_sentence = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Sentence.xlsx")
df_shared_sentence = df_shared_sentence.loc[:,["sentence","frequency"]]
df_shared_sentence.rename(columns={"frequency":"freq_sentence"}, inplace=True)
df_shared_sentence

,sentence,freq_sentence
0,hayır lütfen,7187
1,selam millet,5303
2,elbette hayır,4595
3,aynı şey,2253
4,lütfen hayır,2179
...,...,...
712,tamam kayıt,6
713,mesela aile,6
714,tamam tamam elbette,6
715,mavi kırmızı,6


In [63]:
df_shared_all = pd.concat([df_word, df_shared_twogram, df_shared_threegram, df_shared_fourgram, df_shared_fivegram, df_shared_sentence], axis=1)
df_shared_all

,word,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,mıknatıs,hayır lütfen,11636,hayır hayır lütfen,1498.0,hayır hayır hayır lütfen,165.0,hayır hayır hayır lütfen lütfen,20.0,hayır lütfen,7187.0
1,mısdak,aynı şey,11019,hayır lütfen hayır,392.0,hayır hayır lütfen lütfen,43.0,hayır lütfen lütfen lütfen lütfen,10.0,selam millet,5303.0
2,şey,kadar basit,10150,hayır lütfen lütfen,356.0,hayır lütfen lütfen lütfen,28.0,NaN,NaN,elbette hayır,4595.0
3,şeytan,kadar emin,5919,hayır elbette hayır,345.0,hayır mesela yani tamam,21.0,NaN,NaN,aynı şey,2253.0
4,şeytanî,selam millet,5475,tekrar tekrar aynı,195.0,lütfen hayır hayır hayır,18.0,NaN,NaN,lütfen hayır,2179.0
...,...,...,...,...,...,...,...,...,...,...,...
3411,NaN,yani nefes,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3412,NaN,kadar ifade,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3413,NaN,yani maalesef,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3414,NaN,rahat şarkı,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_shared_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result.xlsx", index=False)